In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models, torchvision.datasets
from torch.utils.data import Dataset
from torchvision.io import read_image
import os
import shutil
import random
import matplotlib.pyplot as plt
from torchvision import transforms

In [11]:
!pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [12]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [13]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from datasets import load_dataset

In [3]:
# Only using 1 parquet file. It contains about 5.8m examples
url = 'https://huggingface.co/datasets/kakaobrain/coyo-700m/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet'

data_files = {"train": url}

pre_train_data = load_dataset("parquet", data_files=data_files, split="train")

In [4]:
pre_train_data

Dataset({
    features: ['id', 'url', 'text', 'width', 'height', 'image_phash', 'text_length', 'word_count', 'num_tokens_bert', 'num_tokens_gpt', 'num_faces', 'clip_similarity_vitb32', 'clip_similarity_vitl14', 'nsfw_score_opennsfw2', 'nsfw_score_gantman', 'watermark_score', 'aesthetic_score_laion_v2'],
    num_rows: 5836073
})

In [5]:
pre_train_data[0]['url']

'https://cdn.shopify.com/s/files/1/0286/3900/2698/products/TVN_Huile-olive-infuse-et-s-227x300_e9a90ffd-b6d2-4118-95a1-29a5c7a05a49_800x.jpg?v=1616684087'

In [6]:
pre_train_data[0]['text']

'Olive oil infused with Tuscany herbs'

In [7]:
pre_train_data = pre_train_data.with_format("torch")

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
# TODO: Need to encode the text descriptions, and clean up images
# TODO: Need to create a final dataset with text, and images
# We can create a custom datalaoder that will load images from urls at runtime.


In [9]:
import requests
from PIL import Image
from io import BytesIO
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class PreTrainDataset(Dataset):
    def __init__(self, dataset, tokenizer, transform=None):
        self.dataset = dataset
        self.transform = transform
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        url = self.dataset[idx]['url']  
        text = self.dataset[idx]['text'] # text has already been encoded and padded 
#         text = self.encode_text(text)
        try:
            response = requests.get(url, timeout=5)
            image = Image.open(BytesIO(response.content)).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, text
        except Exception:
            return None
#     def encode_text(self, example):
#         text = self.tokenizer(example, padding='max_length', max_length=max_seq_len, add_special_tokens=True) # hard-coded max_length for now
#         bos_id = tokenizer.convert_tokens_to_ids("<s>")
#          # add a bos token as well
#         text = {
#             "input_ids": [bos_id] + text["input_ids"],
#             "attention_mask": [1] + text["attention_mask"]
#         }

        return text

In [10]:
import numpy as np

In [11]:
def remove_none_fn(batch):
    batch_without_nones = [item for item in batch if item is not None]
    if not batch_without_nones:
        return []
    if len(batch_without_nones) < len(batch):
        batch_without_nones.extend([batch_without_nones[-1]] * (len(batch)-len(batch_without_nones)))
    images, texts = zip(*batch_without_nones)
    images = torch.stack(images)
    
    tokenized = tokenizer(
        texts,
        padding="longest",
        return_tensors="pt",
        add_special_tokens=True)
    return images, tokenized

In [15]:
custom_transforms = transforms.Compose([
    transforms.Resize((272, 272)),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
])
pre_train_dataset_cleaned = PreTrainDataset(pre_train_data, tokenizer= tokenizer, transform=custom_transforms)
train_loader = DataLoader(pre_train_dataset_cleaned, batch_size=8, shuffle=True, collate_fn=remove_none_fn)

In [16]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [34]:
def train(model, data, lr=0.001, weight_decay=0.00001, num_epochs=20, checkpoint_path='../checkpoints/'):
    
    model.train()

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    epoch = 0

    n = 0

    model = model.to(device)
    train_losses = []
    train_contrastive_losses = []
    train_generative_losses = []
    
    val_losses = []
    val_contrastive_losses = []
    val_generative_losses = []

    batch_size = 2
    while epoch < num_epochs:

        # Using AdamW for now, can try with other optimizers too
       
        optimizer = optim.AdamW(model.parameters(),
                lr=lr,
                weight_decay=weight_decay)
        t_loss = 0
        t_contrastive_loss = 0
        t_generative_loss = 0
        for step, batch in enumerate(data):
            
#             print(batch[0], len(batch[0]))
            # input images, and texts
            if not batch:
                continue
            imgs = batch[0].type(torch.float32).to(device)
            text = batch[1]['input_ids'].type(torch.long).to(device)
#             print(text)

            if len(imgs) < batch_size:
                # Last batch will have less images, text pairs since it will be the
                # remainder of Total images / batch_size.

                # Adjust the learning rate of the last batch by 
                # (size(last_batch) / batch_size) to account 
                # for the smaller size.
                adj_lr = lr * (len(inp) / batch_size)
                optimizer = optim.AdamW(model.parameters(),
                    lr=adj_lr,
                    weight_decay=weight_decay)
            # Since task is to predict next token, the labels will start form position 1
            text_labels = text[:, 1:] 
            total_loss, contrastive_loss, generative_loss = model(imgs, text, text_labels)
            print("-----------------------------------------------------------")
            print(f"Total Loss: {total_loss.item()}   Gen Loss: {generative_loss.item()}   Contr Loss: {contrastive_loss.item()}")
            total_loss.backward()

            optimizer.step()
            optimizer.zero_grad()

            
            # accumulate epoch loss
            t_loss += total_loss
            t_contrastive_loss += contrastive_loss
            t_generative_loss += generative_loss
            del imgs
            del text

        # end of epoch


        epoch += 1

        train_losses.append(t_loss / len(loader))
        train_contrastive_losses.append(t_contrastive_loss / len(loader))
        train_generative_losses.append(t_generative_loss / len(loader))

        epochs.append(epoch)

        val_loss, val_contrastive_loss, val_generative_loss = validation(model, val_data)
        val_losses.append(val_loss)
        val_contrastive_losses.append(val_contrastive_loss)
        val_generative_losses.append(val_generative_loss)
        
        if epoch % 5 == 0: # save model every 5th epoch
            torch.save(model.state_dict(), checkpoint_path.format(epoch))
            
        print("Epoch {}:  Train loss: {}   Train Contrastive Loss: {}   Train Generative Loss: {}]".format(epoch, t_loss / len(loader), t_contrastive_loss / len(loader), t_generative_loss / len(loader)))
        print("Epoch {}:  Val loss: {}   Val Contrastive Loss: {}   Val Generative Loss: {}]".format(epoch, val_loss / len(loader), val_contrastive_loss / len(loader), val_generative_loss / len(loader)))

    return train_losses, train_contrastive_losses, train_generative_losses, val_losses, val_contrastive_losses, val_generative_losses
    

In [31]:
import os
os.chdir("models")

In [35]:
from model import MaMMUT
model = MaMMUT(vocab_size=tokenizer.vocab_size)

In [ ]:
train(model=model, data=train_loader)

-----------------------------------------------------------
Total Loss: 13.47065544128418   Gen Loss: 10.764650344848633   Contr Loss: 2.706005573272705
-----------------------------------------------------------
Total Loss: 16.484277725219727   Gen Loss: 10.535784721374512   Contr Loss: 5.948493003845215
-----------------------------------------------------------
Total Loss: 14.831682205200195   Gen Loss: 10.509672164916992   Contr Loss: 4.322009563446045
-----------------------------------------------------------
Total Loss: 16.234180450439453   Gen Loss: 10.284846305847168   Contr Loss: 5.949333667755127
-----------------------------------------------------------
Total Loss: 17.68054962158203   Gen Loss: 10.13766860961914   Contr Loss: 7.542880535125732
-----------------------------------------------------------
Total Loss: 15.98520278930664   Gen Loss: 9.884836196899414   Contr Loss: 6.100366592407227
-----------------------------------------------------------
Total Loss: 14.952230

-----------------------------------------------------------
Total Loss: 14.017463684082031   Gen Loss: 8.070934295654297   Contr Loss: 5.946529865264893
-----------------------------------------------------------
Total Loss: 14.380477905273438   Gen Loss: 8.43381118774414   Contr Loss: 5.946666240692139
-----------------------------------------------------------
Total Loss: 12.537328720092773   Gen Loss: 7.8917365074157715   Contr Loss: 4.64559268951416
-----------------------------------------------------------
Total Loss: 13.961116790771484   Gen Loss: 8.008559226989746   Contr Loss: 5.95255708694458
-----------------------------------------------------------
Total Loss: 14.284788131713867   Gen Loss: 8.343875885009766   Contr Loss: 5.94091272354126
-----------------------------------------------------------
Total Loss: 13.834314346313477   Gen Loss: 7.8820576667785645   Contr Loss: 5.952256679534912
-----------------------------------------------------------
Total Loss: 14.393145561

In [ ]:
def validation(model, data):
    
    model.eval()

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    epoch = 0

    model.to(device)

    val_loss = 0
    val_contrastive_loss = 0
    val_generative_loss = 0
    
    for step, batch in enumerate(loader):

        # input images, and texts
        imgs = batch[0].type(torch.long).to(device)
        text = batch[1]['input_ids'].type(torch.long).to(device)

        text_labels = text[:, 1:] # labels are the same text just with the <s> token removed
        total_loss, contrastive_loss, generative_loss = model(imgs, text, text_labels)

        val_loss += total_loss
        val_contrastive_loss += contrastive_loss
        val_generative_loss += generative_loss

    return val_loss, val_contrastive_loss, val_generative_loss
    

In [19]:
import torch
import torch.nn as nn

class TextDecoderLayer(nn.Module):
    def __init__(self, 
                d_model, 
                num_heads_mha, 
                num_heads_cross_attn, 
                d_feedforward, 
                d_k,
                d_v, 
                vit_dim):
        super(TextDecoderLayer, self).__init__()

        self.k = nn.Linear(d_model, d_model)
        self.q = nn.Linear(d_model, d_model)
        self.v = nn.Linear(d_model, d_model)

        self.MHA_1 = nn.MultiheadAttention(d_model, num_heads_mha, batch_first =True)
        self.layer_norm1 = nn.LayerNorm(d_model)

        self.k_cross_attn = nn.Linear(vit_dim, d_model)
        self.q_cross_attn = nn.Linear(d_model, d_model)
        self.v_cross_attn = nn.Linear(vit_dim, d_model)

        self.cross_attn = nn.MultiheadAttention(d_model, num_heads_cross_attn, batch_first =True)
        self.layer_norm_cross_attn = nn.LayerNorm(d_model)

        self.fc1 = nn.Linear(d_model, d_feedforward)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(d_feedforward, d_model)
        self.layer_norm_ff = nn.LayerNorm(d_model)

    def forward(self, x, vision_features=None, enable_cross_attn=True, causal_mask=False, padding_mask=None, attn_mask=None):
        """Forward method for decoder layer with added option to disable cross attention and causal masking"""
        k1 = self.k(x)
        q1 = self.q(x)
        v1 = self.v(x)

        out = self.MHA_1(q1, k1, v1, is_causal=causal_mask, key_padding_mask=padding_mask, attn_mask=attn_mask)
#         print(out.shape)
        out = self.layer_norm1(out[0] + x)
        out_layer_norm1 = torch.clone(out)

        if enable_cross_attn:
            k2 = self.k_cross_attn(vision_features)
            q2 = self.q_cross_attn(x)
            v2 = self.v_cross_attn(vision_features)
            out = self.cross_attn(q2, k2, v2)
            out = self.layer_norm_cross_attn(out[0] + out_layer_norm1)
            out_layer_norm_cross_attn = torch.clone(out)
        
        out = self.fc2(self.relu(self.fc1(out)))
        if enable_cross_attn:
            out = self.layer_norm_ff(out + out_layer_norm_cross_attn)
        else:
            out = self.layer_norm_ff(out + out_layer_norm1)

        return out










In [25]:
import torch
import torch.nn as nn
from PIL import Image as PIL_Image
from torchvision.models.vision_transformer import VisionTransformer
from torchvision.transforms import v2
# from text_decoder import TextDecoderLayer
import torch.nn.functional as F

class MaMMUT(nn.Module):
    def __init__(self,
                 image_size: int = 224,
                 patch_size: int = 14,
                 vit_num_layers: int = 32,
                 vit_num_heads: int = 16,
                 vit_hidden_dim: int = 1280,
                 vit_mlp_dim: int = 5120,
                 vit_dropout: float = 0.0, # Potential ablation / extension to add to the replication
                 vit_attention_dropout: float = 0.0, # Potential ablation / extension to add to the replication
                 contrastive_loss_weight: float = 1.0,
                 generative_loss_weight: float = 1.0,
                 text_decoder_depth: int = 6,
                 text_decoder_embed_dim: int = 512,
                 text_decoder_sub_layer_heads: int = 8,
                 text_decoder_feedforward_dim: int = 2048,
                 text_decoder_dk: int = 128,
                 vocab_size: int = 1000,
                 latent_dim: int = 512,
                 contrastive_loss_temp: float = 0.5,
                 contrastive_loss_gamma: float = 1.0):
                 
        super(MaMMUT, self).__init__()   
        self.device = "cuda:0" if torch.cuda.is_available() else "cpu"

        self.vit = VisionTransformer(
            image_size=image_size,
            patch_size=patch_size,
            num_layers=vit_num_layers,
            num_heads=vit_num_heads,
            hidden_dim=vit_hidden_dim,
            mlp_dim=vit_mlp_dim,
            dropout=vit_dropout,
            attention_dropout=vit_attention_dropout,
            num_classes=1000
        )

        self.token_size = vocab_size
        self.text_decoder_embed_dim = text_decoder_embed_dim
        self.text_decoder_sub_layer_heads = text_decoder_sub_layer_heads
        
        self.contrastive_loss_weight = contrastive_loss_weight
        self.generative_loss_weight = generative_loss_weight
        
        self.ifs2tfs = nn.Linear(vit_hidden_dim, text_decoder_embed_dim)
        
        self.text_decoder_depth = text_decoder_depth
        self.text_decoder_layers = []

        self.pos_embedding = nn.Parameter(torch.randn(1, (image_size // patch_size)**2 + 1, vit_hidden_dim))
        
        self.final_layernorm = nn.LayerNorm(self.token_size)

        self.latent_text_features = nn.Linear(text_decoder_embed_dim, text_decoder_embed_dim) # for contrastive loss
        self.pad_token_id = 0 # we can set this in the SentencePiece tokenizer

        self.text_embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=text_decoder_embed_dim, padding_idx=self.pad_token_id)

        self.text_cls_token = nn.Parameter(torch.randn(text_decoder_embed_dim))
        self.contrastive_layernorm = nn.LayerNorm(text_decoder_embed_dim)

        self.loss_criterion = nn.CrossEntropyLoss(ignore_index=self.pad_token_id).to(self.device)
        self.contrastive_loss_temp = contrastive_loss_temp
        self.contrastive_loss_gamma = contrastive_loss_gamma
        self.image_size = image_size
        self.patch_size = patch_size
        
        
        # Changing logic for the decoder layer. This way we can disable cross-attention during the forward pass and keep everything else the same
        for i in range(text_decoder_depth):
            self.text_decoder_layers.append(TextDecoderLayer(d_model=text_decoder_embed_dim, num_heads_mha=text_decoder_sub_layer_heads, \
                                                            num_heads_cross_attn=text_decoder_sub_layer_heads, d_feedforward=text_decoder_feedforward_dim, \
                                                             d_k=text_decoder_dk, d_v=(text_decoder_embed_dim // text_decoder_sub_layer_heads), vit_dim=vit_hidden_dim).to(self.device)
                                                             )
        self.decoder_output_features_to_text_tokens_layer = nn.Linear(self.text_decoder_embed_dim, self.token_size) # for captioning loss
            
        
    def cropped_positional_encoding(self, feats):
        # feats shape: N x (H_p x W_p) x Hidden
        n, h_w, hidden = feats.shape


        # take out cls token before upsampling
        cls_pos_embed = self.pos_embedding[:, 0, :]
        pos_embeddings = self.pos_embedding[:, 1: :]

        pos_embeddings = pos_embeddings.reshape(1, self.image_size // self.patch_size, self.image_size // self.patch_size, hidden).permute(0, -1, 1, 2)

        # Upsample using bilinear interpolation
        upsample_layer = nn.Upsample(mode='bilinear', scale_factor=4)
        upsampled_pos_embeddings = upsample_layer(pos_embeddings)
        random_crop = v2.RandomCrop(pos_embeddings.shape[2])
        cropped_pos_encoding = random_crop(upsampled_pos_embeddings)

        # cropped_pos_encoding shape: N x (H_p x W_p) x Hidden. Reshape to align with feats
        cropped_pos_encoding = cropped_pos_encoding.reshape(1, h_w-1, hidden)
        
        cropped_pos_encoding = torch.cat([cropped_pos_encoding, cls_pos_embed.reshape(1, 1, hidden)], dim=1)


        return feats + cropped_pos_encoding

        
    def get_vision_features(self, img: torch.tensor):
        # image has shape N x C x H x W where
        # N is the batch size
        # C is the channel size
        # H is the image height
        # W is the image width
#         preprocessing = v2.Compose([
#             v2.ToImage(),
#             v2.Resize((272,272)),
#             v2.RandomCrop(224)
#         ])

#         img = PIL_Image.open("example_2353642598754.jpeg")
#         img = preprocessing(img)

        # Add batch dimension - for testing on one image, remove for training
#         img = img.unsqueeze(0)
        # (n, c, h, w) -> (n, hidden_dim, n_h, n_w), converts into patches
        feats = self.vit._process_input(img)
        # Expand the CLS token to the full batch
        batch_class_token = self.vit.class_token.expand(img.shape[0], -1, -1)
        feats = torch.cat([batch_class_token, feats], dim=1)
        
        feats = self.cropped_positional_encoding(feats)

        feats = self.vit.encoder(feats)

        # Fetch pre-prended CLS token at position 0 in dimension 1
        feats = feats[:, 0].reshape(feats.shape[0], 1, feats.shape[-1])
                
        return feats
    
    def img_feat_size_to_txt_feat_size(self, vision_features: torch.tensor):
        return self.ifs2tfs(vision_features)
    
    def contrastive_text_features(self, text_embeds: torch.Tensor):
        # text has shape N x S
        # Remember to pass bidirectional mask (as far as I understand, a mask that allows attention to all non-padded areas or maybe just all non-CLS areas and maybe stops cls from attending to padding TODO: Clarify)
        # Remember to perform residual additions         
        # expand to match dimensions
        cls_tokens = self.text_cls_token.expand(text_embeds.shape[0], 1, self.text_decoder_embed_dim).to(self.device)
        # Add cls tokens to start of the sequences
        text_embeds = torch.cat([cls_tokens, text_embeds], dim=1)
        cls_padding_mask = (text_embeds == 0).all(dim=-1) # From nn.Embedding, padding tokens are embedded as vector of 0s. Result should be shape N x S.

        output = text_embeds.clone()
        for i, layer in enumerate(self.text_decoder_layers):
            # Disable cross-attention for contrastive features
            output = layer(output, enable_cross_attn=False, padding_mask=cls_padding_mask)

        output = output[:, 0]
        output = self.contrastive_layernorm(output)
        return output
    
    def generative_text_features(self, text_embeds: torch.tensor, vision_features: torch.tensor):
        # Remember to toggle causal in forward pass
        # Remember to perform residual additions

        attn_mask = torch.triu(torch.ones((text_embeds.shape[1], text_embeds.shape[1])), diagonal=1).bool().to(self.device) # Assuming shape[1] is the sequence dim
        output = text_embeds.clone()
        padding_mask = (text_embeds == 0).all(dim=-1)
        for i, layer in enumerate(self.text_decoder_layers):
            # Disable cross-attention for odd numbered layers
            if i % 2 != 0:
                output = layer(output, vision_features=None, enable_cross_attn=False, causal_mask=True, attn_mask=attn_mask, padding_mask=padding_mask)
            else:
                # enable cross-attention for even numbered layers
                output = layer(output, vision_features=vision_features, enable_cross_attn=True, causal_mask=True, attn_mask=attn_mask, padding_mask=padding_mask)
        return output

    
    def contrastive_loss(self, vision_features: torch.tensor, constrastive_text_features: torch.tensor):
        """Implement Focal-contrastive loss as in the paper"""
        similarity = (vision_features @ constrastive_text_features.T) / self.contrastive_loss_temp
        similarity = similarity.squeeze(1)
        # In contrastive learning we aim to minimize loss for between the matching image and text pairs, and maximize loss 
        # for mismatching image text pairs.
        # after the matrix multipication, shape will be N x N
        # each row represents image i, and each column would represent each caption
        # Therefore, the matching pairs will be across the diagonal (0,0), (1, 1) ... and we can treat this as a classification task
        # where we compute the loss between the text_logits and its matching image and vice-versa for the image loss
    
        # We can construct the labels by just creating a diagonal matrix
        labels = torch.arange(similarity.shape[0]).to(self.device)
        labels_one_hot = F.one_hot(labels, num_classes=similarity.shape[0]).to(self.device)
        probs_imgs = F.softmax(similarity, dim=1) # using softmax instead of sigmoid
        p_t_imgs = torch.sum(labels_one_hot * probs_imgs, dim=1)
        p_t_imgs = torch.clamp(p_t_imgs, min=1e-5)
        loss_i2t = -(((1 - p_t_imgs) ** self.contrastive_loss_gamma) * (torch.log(p_t_imgs))).mean()
        probs_texts = F.softmax(similarity, dim=0)
        p_t_texts = torch.sum(labels_one_hot * probs_texts, dim=1)
        p_t_texts = torch.clamp(p_t_texts, min=1e-5)

        loss_t2i = -(((1 - p_t_texts) ** self.contrastive_loss_gamma) * (torch.log(p_t_texts))).mean()
        total_contrastive_loss = (loss_i2t + loss_t2i) / 2

        return total_contrastive_loss.to(self.device)


    def generative_loss(self, generative_text_features: torch.tensor, text_labels: torch.tensor):
        generative_text_features = generative_text_features.permute(0, -1, 1) # cross-entropy expects N x C as first two dims
        loss = self.loss_criterion(generative_text_features, text_labels)
        return loss.to(self.device)

    def decoder_output_features_to_text_tokens(self, text_features: torch.tensor):
        return self.final_layernorm(self.decoder_output_features_to_text_tokens_layer(text_features))
        
    
    def forward(self, image, text, text_labels):
        # Pseudocode for now, need to fully implement and test
        # TODO: Implement average pooling over spatial dimension and sequence where appropriate
        # TODO: Add tokenizer & params ------- Tokenizer would be added in training pipeline
        text_embeds = self.text_embeddings(text)
#         print(text_embeds)
        vision_features = self.get_vision_features(image)
        vision_features_contrastive = self.img_feat_size_to_txt_feat_size(vision_features) # projects image feature dim to text feature dim
        
        constrastive_text_features = self.contrastive_text_features(text_embeds)
        constrastive_text_features = self.latent_text_features(constrastive_text_features)

        contrastive_loss = self.contrastive_loss(vision_features_contrastive, constrastive_text_features)
        
        # Since task will be to generate next token, we only go up until the second last token
        generative_text_features = self.generative_text_features(text_embeds[:, :-1], vision_features)

        text_logits = self.decoder_output_features_to_text_tokens(generative_text_features)

        generative_loss = self.generative_loss(text_logits, text_labels)
        loss = self.contrastive_loss_weight * contrastive_loss + self.generative_loss_weight * generative_loss
        print("generative_loss", generative_loss.item())
        print("contrastive_loss", contrastive_loss.item())
        return loss, contrastive_loss, generative_loss

In [26]:
model = MaMMUT(vocab_size=tokenizer.vocab_size)

In [11]:
!pwd

/home/hice1/hfaisal8/CS7643/project/RepliMaMMUT
